In [1]:
import faro
import faro.proto.proto_types as pt

In [2]:
class FaroOptions(object):
    def __init__(self):
        self.detect_port = faro.DEFAULT_PORT
        self.rec_port = faro.DEFAULT_PORT
options = FaroOptions()
client = faro.FaceClient(options)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Connect Failed"
	debug_error_string = "{"created":"@1555872432.083500000","description":"Failed to create subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":2715,"referenced_errors":[{"created":"@1555872432.083497000","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":456,"referenced_errors":[{"created":"@1555872432.083230000","description":"Connect Failed","file":"src/core/ext/filters/client_channel/subchannel.cc","file_line":963,"grpc_status":14,"referenced_errors":[{"created":"@1555872432.083198000","description":"Failed to connect to remote host: Connection refused","errno":61,"file":"src/core/lib/iomgr/tcp_client_posix.cc","file_line":207,"os_error":"Connection refused","syscall":"connect","target_address":"ipv4:127.0.0.1:50030"}]}]}]}"
>

This next command will contact the service and return some basic information about the currently running service.  If successful this should return a "status: READY" message.

In [ ]:
status = client.status()[1]
status

Now use pyvision to load an image.

In [ ]:
import pyvision as pv
lena = pv.Image(pv.LENA)
lena.show()

In [ ]:
faces = client.detect(lena)
# This should also print a log message on the server display.
faces

In [ ]:
faces

In [ ]:
# Prototypes offers converters for common datastructures.  This one converts from a protobuf to a pyvision (pv) rect.
rect = pt.rect_proto2pv(faces.face_records[0].detection.location)
rect

In [ ]:
lena.annotateThickRect(rect)
lena.show()

Now lets try the same thing but with the mirrored image.

In [ ]:
cvim = lena.asOpenCV2()[:,::-1,:]
lena_mirror = pv.Image(cvim)
#lena_mirror = pv.AffineScale(0.99,lena.size)(lena_mirror)
lena_mirror.show()

In [ ]:
faces_mirror = client.detect(lena_mirror)
rect_mirror = pt.rect_proto2pv(faces_mirror.face_records[0].detection.location)
lena_mirror.annotateThickRect(rect_mirror)
lena_mirror.show()

In [ ]:
faces = client.extract(lena,faces)
faces_mirror = client.extract(lena_mirror,faces)

In [ ]:
# Here we can decode the template and view it as a vector.
pt.vector_proto2np( faces.face_records[0].template.data )

Now we compare the to face templates.


In [ ]:
all_rec = list(faces.face_records) + list( faces_mirror.face_records)

scores = client.score(faces.face_records,faces_mirror.face_records)

scores

Now check if it is a match.

In [ ]:
print('Score:',scores[0,0],' Thresh:',status.match_threshold)
if scores[0,0] < status.match_threshold:
    print("The faces MATCH.")
else:
    print("The faces DO NOT MATCH.")

The lena and lena_mirror images typically produce scores near the 0.6 threshold.  The dlib introduces random perturbations to the images so sometimes the faces match and sometimes they don't.  Most other algorithms are determinisitic and will produce the same value each time.